In [11]:
import pandas as pd

df = pd.read_csv('Data/download/ratings.csv')
df

,user_uid,element_uid,rating,ts
0,571252,1364,10,4.430517e+07
1,63140,3037,10,4.430514e+07
2,443817,4363,8,4.430514e+07
3,359870,1364,10,4.430506e+07
4,359870,3578,9,4.430506e+07
...,...,...,...,...
438785,170707,1539,8,4.173086e+07
438786,32659,6127,8,4.173085e+07
438787,353752,3336,8,4.173079e+07
438788,492350,7984,7,4.173078e+07


Проверяем, есть ли среди пользователей те, кто оценил один фильм дважды. Таких пользователей нет, можно работать с данными без группировки и учёта времени.

In [26]:
# Проверка наличия хотя бы двух разных оценок в колонке "rating" в каждой группе
grouped_df = df.groupby(["user_uid", "element_uid"]).filter(lambda x: x["rating"].nunique() >= 2)

# Вывод пользователей, у которых есть хотя бы две разные оценки одному фильму
print("Пользователи с двумя разными оценками одному фильму:")
print(grouped_df["user_uid"].unique())
del grouped_df


Пользователи с двумя разными оценками одному фильму:
[]
